# Build an app for public health

## <i>Data cleaning</i>

My mission here for <strong>Santé publique France</strong> is to create an application related to food from the [Open Food Fact dataset](https://world.openfoodfacts.org/).<br>

I want to offer a French application which, when entering the product or product category of your choice, will offer a top 3 of the most recommended brands in terms of nutritional quality.<br>

The app name is : <i><strong>Buy Better</strong></i>
<br><br>

<u><strong>Summary</strong></u> :<br>
<br>
<strong>Tools </strong> [(here)](#another_cell_0)<br>
 
<strong>I. <i>Buy better</i> : data cleaning and augmentation</strong><br>
     -- A.[food_data.csv](http://localhost:8888/edit/food_data.csv) : dataset description and cleaning [(here)](#another_cell_a)<br>
     -- B. <i>Buy better</i> : data selection [(here)](#another_cell_b)<br>
     -- C. <i>Buy better</i> : data augmentation [(here)](#another_cell_c)<br>
     ---- <i>a. Qualitative data normalizing</i> [(here)](#another_cell_d)<br>
     ---- <i>b. Quantative data augmentation using linear regression</i> [(here)](#another_cell_e)<br>
     ---- <i>c. Quantative data augmentation using iterative imputer</i> [(here)](#another_cell_f)<br>
     ---- <i>d. Quantative data augmentation using fillna</i> [(here)](#another_cell_g)<br>
     -- D. <i>Buy better</i> : anomaly detection and treatment [(here)](#another_cell_h)<br><br>
<strong>Conclusion (part 1) </strong> [(here)](#another_cell_i)


<a id='another_cell_0'></a>

### Tools

In [270]:
from sklearn.experimental import enable_iterative_imputer

In [271]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import warnings
import re
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.impute import IterativeImputer
from sklearn.ensemble import IsolationForest
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [272]:
# Check null values function :
def check_nan(data, column):
    if len(data[data[column].isnull()]) == 0:
        print ("The column", column, "does not contain any missing values")
    else :
        print("The column", column, "contain ", len(data[data[column].isnull()]), " missing values")
        
# Check duplicates function :
def check_duplicates(data, column):
    n = len(data.index)
    if len(data[column].drop_duplicates()) == n :
        print ("The column", column, "does not contain duplicates")
    else :
        print("The column", column, "contains ", len(data[data[column].duplicated()]), " duplicates.")
        return data[data[column].duplicated()]
    
# Determine if the parameter value is missing
def num_missing(x):
    return sum(x.isnull())

# Using the Interquartile rule to find and count outliers
def count_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    outliers = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        lower_bound = q1[col] - 1.5 * iqr[col]
        upper_bound = q3[col] + 1.5 * iqr[col]
        num_outliers = len(df[(df[col] < lower_bound) | (df[col] > upper_bound)])
        outliers[col] = num_outliers
    return outliers

## I. <i>Buy better</i> : data cleaning and augmentation

<a id='another_cell_a'></a>

### A. [food_data.csv](http://localhost:8888/edit/food_data.csv) : dataset description and cleaning

- <strong>Data set loading and description</strong>

In [273]:
data = pd.read_csv('food_data.csv', sep='\t', low_memory=False)

In [109]:
data.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_fr,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_fr,main_category,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","A compléter,Informations nutritionnelles à com...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,NaN,NaN,NaN,NaN,28 g (1 ONZ),NaN,0.0,[ bananas -> en:bananas ]

In [274]:
data.shape

(320772, 162)

In [275]:
# Columns labels
print("Columns labels :", data.columns.tolist())

Columns labels : ['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_fr', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_fr', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_fr', 'ingredients_text', 'allergens', 'allergens_fr', 'traces', 'traces_tags', 'traces_fr', 'serving_size', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_fr', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutrition_grade_uk', 'nutrition_grade_fr', 

- <strong>Check null and duplicates values on primarey key</strong>

In [276]:
check_nan(data, "code")

The column code contain  23  missing values


<div class="alert-danger">
☛ We can delete these rows from the dataframe
</div>

In [277]:
# Get names of indexes for rows to drop
indexNames = data[data['code'].isna()].index
# Delete these row indexes from original dataFrame
data.drop(indexNames , inplace=True)
data.shape

(320749, 162)

In [278]:
check_duplicates(data, "code")

The column code does not contain duplicates


<div class="alert-success">
    <u>Notice</u> :
<i><li> The "code" column represent the products' ids
<li> It seems to be the primarey key here
<li> We count 320 749 unique codes in this dataframe
</li></i></div>

- <strong>Check null and unique values on products' names</strong>

In [279]:
check_nan(data, "product_name")
check_nan(data, "generic_name")

The column product_name contain  17762  missing values
The column generic_name contain  267976  missing values


<div class="alert-warning">
☛ The column "product_name" seems to be appropriate to give us products name information
</div>

In [280]:
# Check rows without products name
data[data["product_name"].isna()].shape

(17762, 162)

<div class="alert-danger">
☛ We can not use data without product name, so we can delete these rows from the dataframe
</div>

In [281]:
# Get names of indexes for rows to drop
indexNames_ = data[data['product_name'].isna()].index
# Delete these row indexes from original dataFrame
data.drop(indexNames_ , inplace=True)
data.shape

(302987, 162)

<div class="alert-success">
    <u> Notice</u> :<br>
Our application is intended for a French audience, therefore the products offered by the application must all be available for purchase in France.
</div>

- <strong>Filter our dataframe on French branded products</strong>

In [282]:
# List of countries
print("List of countries :", data['countries_fr'].unique())

List of countries : ['France' 'États-Unis' 'Royaume-Uni' 'France,États-Unis' 'Canada' 'Suisse'
 'Espagne' 'Allemagne' 'Slovénie' 'Italie' 'Roumanie' 'Australie'
 'France,Royaume-Uni' 'Belgique,France,Pays-Bas,Royaume-Uni' 'Guadeloupe'
 'Royaume-Uni,États-Unis' 'Canada,États-Unis' 'Pays-Bas'
 'Australie,France' 'Canada,France,Suisse,États-Unis'
 'France,États-Unis,en:Etats-unis' 'Irlande' 'France,La Réunion'
 'Australie,Suisse' 'France,Allemagne' 'Brésil' 'Belgique,France'
 'États-Unis,en:Australie' 'Arabie saoudite' 'Émirats arabes unis'
 'Moldavie' 'Belgique' nan 'Singapour' 'Finlande' 'Australie,États-Unis'
 'France,Suisse'
 'Australie,Nouvelle-Zélande,Singapour,Thaïlande,États-Unis' 'Panama'
 'Costa Rica' 'États-Unis,en:Pays-bas' 'Espagne,Royaume-Uni,en:Scotland'
 'Thaïlande' 'Mexique' 'Chine' 'Luxembourg' 'Liban' 'Suisse,États-Unis'
 'États-Unis,en:Deutschland' 'France,Suisse,États-Unis'
 'Belgique,Suisse,États-Unis' 'Danemark,États-Unis,en:Spanien'
 'États-Unis,en:Belgie,en:Nederl

In [283]:
# New df only on with French products
data_fr = data[data["countries_fr"].str.contains(pat = "France", na=False)]
print("List of countries including France :", data_fr["countries_fr"].unique())

List of countries including France : ['France' 'France,États-Unis' 'France,Royaume-Uni'
 'Belgique,France,Pays-Bas,Royaume-Uni' 'Australie,France'
 'Canada,France,Suisse,États-Unis' 'France,États-Unis,en:Etats-unis'
 'France,La Réunion' 'France,Allemagne' 'Belgique,France' 'France,Suisse'
 'France,Suisse,États-Unis' 'France,Italie'
 'France,Royaume-Uni,États-Unis' 'Australie,France,Royaume-Uni'
 'Canada,France' 'Canada,France,Suisse' 'Belgique,Canada,France,Suisse'
 'France,Saint-Pierre-et-Miquelon' 'France,Russie,Espagne,Royaume-Uni'
 'France,Suisse,Royaume-Uni' 'France,Nouvelle-Zélande'
 'Belgique,France,Royaume-Uni' 'Canada,France,États-Unis'
 'France,Pays-Bas' 'France,Pays-Bas,Royaume-Uni' 'France,Qatar'
 'Algérie,France' 'France,Guyane' 'Belgique,France,Pays-Bas'
 'France,Tunisie' 'Belgique,France,Suisse'
 'Belgique,France,Saint-Pierre-et-Miquelon' 'France,en:An' 'France,Japon'
 'France,Inde' 'France,Portugal,Royaume-Uni'
 'France,Guyane,Grèce,Guadeloupe,Suriname,Royaume-Uni'
 'Ch

In [284]:
data_fr.shape

(91247, 162)

- <strong>Check null values on brands' names</strong>

In [285]:
# Check rows without brands
data_fr[data_fr["brands"].isna()].shape

(5849, 162)

<div class="alert-danger">
☛ For this project, we need the "brands" column to be filled in, so we can remove the blank rows in it
</div>

In [286]:
# Get names of indexes for rows to drop
_indexnames_ = data_fr[data_fr['brands'].isna()].index
# Delete these row indexes from original dataFrame
data_fr.drop(_indexnames_ , inplace=True)
data_fr.shape

C:\Users\hp\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(85398, 162)

- <strong>Check and drop empty columns</strong>

In [287]:
# Apply num_missing function for each column
print("Missing values per column :", "\n")
print(data_fr.apply(num_missing, axis=0))

Missing values per column : 

code                                              0
url                                               0
creator                                           0
created_t                                         0
created_datetime                                  1
last_modified_t                                   0
last_modified_datetime                            0
product_name                                      0
generic_name                                  47363
quantity                                      17433
packaging                                     28736
packaging_tags                                28734
brands                                            0
brands_tags                                       6
categories                                    24571
categories_tags                               24571
categories_fr                                 24571
origins                                       70182
origins_tags                      

In [288]:
# Create boolean mask for columns with all missing values
missing_columns_mask = data_fr.apply(num_missing, axis=0) == len(data_fr)

# Filter columns that have no values at all
missing_columns = data_fr.columns[missing_columns_mask]
print("Columns with no values at all :", "\n")
print(missing_columns)

Columns with no values at all : 

Index(['cities', 'allergens_fr', 'no_nutriments', 'ingredients_from_palm_oil',
       'ingredients_that_may_be_from_palm_oil', 'nutrition_grade_uk',
       'butyric-acid_100g', 'caproic-acid_100g', 'caprylic-acid_100g',
       'capric-acid_100g', 'myristic-acid_100g', 'palmitic-acid_100g',
       'stearic-acid_100g', 'behenic-acid_100g', 'lignoceric-acid_100g',
       'cerotic-acid_100g', 'melissic-acid_100g',
       'dihomo-gamma-linolenic-acid_100g', 'elaidic-acid_100g',
       'gondoic-acid_100g', 'mead-acid_100g', 'erucic-acid_100g',
       'nervonic-acid_100g', 'chlorophyl_100g', 'glycemic-index_100g',
       'water-hardness_100g'],
      dtype='object')


<div class="alert-danger">
☛ We can drop all this empty columns
</div>

In [289]:
# Remove all null value column
data_fr.dropna(how='all', axis=1, inplace=True)

<ipython-input-289-4eacf910730b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fr.dropna(how='all', axis=1, inplace=True)


In [290]:
data_fr.shape

(85398, 136)

<a id='another_cell_b'></a>

### B. <i>Buy better</i> : data selection

- <strong>Data selection on variables which have more than 50% complete values</strong>

In [291]:
# Create boolean mask for columns which have less than 50% of missing values
half_full_columns_mask = data_fr.apply(num_missing, axis=0) < len(data_fr) / 2

# Filter columns that have less than 50% of missing values
half_full_columns = data_fr.columns[half_full_columns_mask]
print("Half-full columns labels :", "\n")
print(half_full_columns, "\n")
print("Number of columns concerned :", len(half_full_columns))

Half-full columns labels : 

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name', 'quantity',
       'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories',
       'categories_tags', 'categories_fr', 'purchase_places', 'countries',
       'countries_tags', 'countries_fr', 'ingredients_text', 'additives_n',
       'additives', 'ingredients_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil_n', 'nutrition_grade_fr',
       'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_fr',
       'main_category', 'main_category_fr', 'image_url', 'image_small_url',
       'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g',
       'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'nutrition-score-fr_100g', 'nutrition-score-uk_100g'],
      dtype='object') 

Number of columns concerned : 46


<div class="alert-success">
    For the project, <strong>we can select the following data</strong> :
<br>
- <strong>product_name</strong> (our primary key),<br>
- <strong>brands</strong>,<br> 
- <strong>main_category_fr</strong>,<br> 
- <strong>nutrition_grade_fr</strong><br> 
- <strong>additives_n</strong> (to discriminate against products with a lot of additives),<br> 
- <strong>saturated_fat_100g and ingredients_from_palm_oil_n</strong> (to discriminate against products with),<br>
- <strong>fiber_100g, proteins_100g and fat_100g</strong> (positively weighted in the calculation of the nutriscore),<br>
- <strong>energy_100g and carbohydrates_100g</strong> (negatively weighted in the calculation of the nutriscore).
</div>

- <strong>Data selection on variables which have less than 50% complete values</strong>

<div class="alert-warning">
☛ These variables have little data, so we need to select as few as possible

In [292]:
# Create boolean mask for columns which have more than 50% of missing values
half_empty_columns_mask = data_fr.apply(num_missing, axis=0) > len(data_fr) / 2

# Filter columns that have more than 50% of missing values
half_empty_columns = data_fr.columns[half_empty_columns_mask]
print("Half-empty columns labels :", "\n")
print(half_empty_columns, "\n")
print("Number of columns concerned :", len(half_empty_columns))

Half-empty columns labels : 

Index(['generic_name', 'origins', 'origins_tags', 'manufacturing_places',
       'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_fr',
       'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo',
       'cities_tags', 'stores', 'allergens', 'traces', 'traces_tags',
       'traces_fr', 'serving_size', 'additives_tags', 'additives_fr',
       'ingredients_from_palm_oil_tags',
       'ingredients_that_may_be_from_palm_oil_tags', 'energy-from-fat_100g',
       'lauric-acid_100g', 'arachidic-acid_100g', 'montanic-acid_100g',
       'monounsaturated-fat_100g', 'polyunsaturated-fat_100g',
       'omega-3-fat_100g', 'alpha-linolenic-acid_100g',
       'eicosapentaenoic-acid_100g', 'docosahexaenoic-acid_100g',
       'omega-6-fat_100g', 'linoleic-acid_100g', 'arachidonic-acid_100g',
       'gamma-linolenic-acid_100g', 'omega-9-fat_100g', 'oleic-acid_100g',
       'trans-fat_100g', 'cholesterol_100g', 'sucrose_100g', 'glucose_100g',
       'fructos

<div class="alert-success">
For the project, <strong>we can select the following data</strong> :
<li><strong>vitamin-d_100g and vitamin-c_100g</strong> (positively weighted in the calculation of the nutriscore)</li>   
<li><strong>alcohol_100g and polyols_100g</strong> (negatively weighted in the calculation of the nutriscore)</li>
</div>

- <strong><i>Buy better</i> new dataframe</strong>

In [293]:
# By better new df
data_bb = data_fr[['product_name','main_category_fr','brands','nutrition_grade_fr',
    'fiber_100g','proteins_100g','fat_100g',
    'energy_100g','alcohol_100g','carbohydrates_100g','polyols_100g',
    'saturated-fat_100g','ingredients_from_palm_oil_n','additives_n','vitamin-d_100g','vitamin-c_100g']]
data_bb.shape

(85398, 16)

In [294]:
data_bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85398 entries, 0 to 320770
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name                 85398 non-null  object 
 1   main_category_fr             60827 non-null  object 
 2   brands                       85398 non-null  object 
 3   nutrition_grade_fr           60861 non-null  object 
 4   fiber_100g                   45362 non-null  float64
 5   proteins_100g                63695 non-null  float64
 6   fat_100g                     47049 non-null  float64
 7   energy_100g                  63967 non-null  float64
 8   alcohol_100g                 2327 non-null   float64
 9   carbohydrates_100g           46619 non-null  float64
 10  polyols_100g                 301 non-null    float64
 11  saturated-fat_100g           61802 non-null  float64
 12  ingredients_from_palm_oil_n  53266 non-null  float64
 13  additives_n    

<div class="alert-success">
    <u> Notice</u> :<br>
We have 4 qualitative variables and 12 quantitative variables.
</div>

<a id='another_cell_c'></a>

### C. <i>Buy better</i> : data augmentation

<a id='another_cell_d'></a>

#### a. <i>Buy better</i> dataframe : qualitative data normalizing

In [295]:
# Random value check
print("Random product names :", data_bb["product_name"].sample(5), "\n")
print("Random product categories :", data_bb["main_category_fr"].sample(5), "\n")
print("Random product brands :", data_bb["brands"].sample(5), "\n")
print("Random nutrition grades :", data_bb["nutrition_grade_fr"].sample(5), "\n")

Random product names : 225154                 Farine de blé bio
191830    Terrine de campagne aux cèpes 
276136               Kids tomates bonbon
177048                          goodycao
200748            Saucisses de Francfort
Name: product_name, dtype: object 

Random product categories : 228344                                   Boissons
292116                                  Chocolats
185800                                  Volailles
247663                                        NaN
316420    Aliments et boissons à base de végétaux
Name: main_category_fr, dtype: object 

Random product brands : 186130         Aldi
179658       Deluxe
198942        Pural
190138      Panzani
236695    carrefour
Name: brands, dtype: object 

Random nutrition grades : 222047      c
195138      c
194715    NaN
251892      d
194046      d
Name: nutrition_grade_fr, dtype: object 



<div class="alert-danger">
☛ Double spaces and special characters should be removed, and NaN values replace by "Unknown"
</div>

In [296]:
columns_quali_to_fill = ['product_name', 'main_category_fr', 'brands', 'nutrition_grade_fr']

# Replace NaN values by "Unknown" in our qualitative variables
for column in columns_quali_to_fill:
    data_bb[column].fillna('Unknown', inplace=True)

C:\Users\hp\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [297]:
# Remove special characters
for column in columns_quali_to_fill:
    data_bb[column] = data_bb[column].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

<ipython-input-297-3f9f5a8b41ce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bb[column] = data_bb[column].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))


In [298]:
# Put in lowercase and remove double spaces
for column in columns_quali_to_fill:
    data_bb[column] = data_bb[column].str.lower().replace('  ', ' ')

<ipython-input-298-08adaf9be986>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bb[column] = data_bb[column].str.lower().replace('  ', ' ')


In [299]:
# Check qualitative values
print("Check product names :", data_bb["product_name"].unique(), "\n")
print("Check product categories :", data_bb["main_category_fr"].unique(), "\n")
print("Check product brands :", data_bb["brands"].unique(), "\n")
print("Check nutrition grades :", data_bb["nutrition_grade_fr"].unique(), "\n")

Check product names : ['farine de blé noir' 'lion peanut x2' 'pack de 2 twix' ...
 'roussette du bugey 2011' 'rillette doie' 'tomates aux vermicelles '] 

Check product categories : ['unknown' 'aliments et boissons à base de végétaux' 'sablés' ...
 'encremesvegetalesabasedecocopourcuisiner' 'enmaltvinegar' 'attieke'] 

Check product brands : ['ferme ty rnao' 'sunridge' 'twix lundberg' ... 'l boulaire'
 'roussette du bugey' 'sans marquedlambert'] 

Check nutrition grades : ['unknown' 'e' 'd' 'c' 'a' 'b'] 



<a id='another_cell_e'></a>

#### b. <i>Buy better</i> dataframe : quantative data augmentation using linear regression

In [300]:
data_bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85398 entries, 0 to 320770
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name                 85398 non-null  object 
 1   main_category_fr             85398 non-null  object 
 2   brands                       85398 non-null  object 
 3   nutrition_grade_fr           85398 non-null  object 
 4   fiber_100g                   45362 non-null  float64
 5   proteins_100g                63695 non-null  float64
 6   fat_100g                     47049 non-null  float64
 7   energy_100g                  63967 non-null  float64
 8   alcohol_100g                 2327 non-null   float64
 9   carbohydrates_100g           46619 non-null  float64
 10  polyols_100g                 301 non-null    float64
 11  saturated-fat_100g           61802 non-null  float64
 12  ingredients_from_palm_oil_n  53266 non-null  float64
 13  additives_n    

<div class="alert-success">
E kcal is related to fat, alcohol, proteins, carbohydrates, polyols and fiber<br>
☛ <strong>We can replace the missing values ​​of the "energy_100g" variable using linear regression to predict them.</strong>
</div>

In [304]:
# Data preparation
df = data_bb[['energy_100g', 'fat_100g', 'alcohol_100g', 'proteins_100g',
           'carbohydrates_100g', 'polyols_100g', 'fiber_100g']]

In [305]:
# Replace Nan values by 0 in our related variables
columns_to_fill = ['fat_100g', 'alcohol_100g', 'proteins_100g', 'carbohydrates_100g', 'polyols_100g', 'fiber_100g']

for i in columns_to_fill:
    data_bb[i].fillna(0.0, inplace=True)

In [306]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85398 entries, 0 to 320770
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   energy_100g         63967 non-null  float64
 1   fat_100g            85398 non-null  float64
 2   alcohol_100g        85398 non-null  float64
 3   proteins_100g       85398 non-null  float64
 4   carbohydrates_100g  85398 non-null  float64
 5   polyols_100g        85398 non-null  float64
 6   fiber_100g          85398 non-null  float64
dtypes: float64(7)
memory usage: 5.2 MB


In [307]:
# Linear regression
reg_multi = smf.ols('energy_100g~fat_100g+alcohol_100g+proteins_100g+carbohydrates_100g+polyols_100g+fiber_100g', data=df).fit()

# Prediction
df_na = df[df["energy_100g"].isnull()]
predict_na = reg_multi.predict(df_na)

In [308]:
# Imputations
data_bb["energy_100g"][data_bb["energy_100g"].isnull()] = predict_na
df["energy_100g"][df["energy_100g"].isnull()] = predict_na

<ipython-input-308-5233440cda25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bb["energy_100g"][data_bb["energy_100g"].isnull()] = predict_na
C:\Users\hp\anaconda3\lib\site-packages\pandas\core\series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85398 entries, 0 to 320770
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   energy_100g         85398 non-null  float64
 1   fat_100g            85398 non-null  float64
 2   alcohol_100g        85398 non-null  float64
 3   proteins_100g       85398 non-null  float64
 4   carbohydrates_100g  85398 non-null  float64
 5   polyols_100g        85398 non-null  float64
 6   fiber_100g          85398 non-null  float64
dtypes: float64(7)
memory usage: 7.2 MB


<a id='another_cell_f'></a>

#### b. <i>Buy better</i> dataframe : quantative data augmentation using iterative imputer

In [310]:
# Random value check
print("Random saturated fat :", data_bb["saturated-fat_100g"].sample(10), "\n")
print("Random ingredients from palm oil :", data_bb["ingredients_from_palm_oil_n"].sample(10), "\n")

Random saturated fat : 228795     9.9
228406     NaN
207620     4.3
240519     0.2
252176     NaN
187266     NaN
179479    23.0
198702     3.0
225198     NaN
226646    17.0
Name: saturated-fat_100g, dtype: float64 

Random ingredients from palm oil : 263955    NaN
251077    0.0
196979    0.0
211246    0.0
226295    0.0
233750    1.0
260444    0.0
219714    0.0
207033    0.0
208893    0.0
Name: ingredients_from_palm_oil_n, dtype: float64 



<div class="alert-success">
We know that palm oil products automatically contain saturated fat, palm oil being a type of saturated fat.<br>
☛ <strong>We can use the Iterative impute method to estimate the missing values of the variable "saturated-fat_100g" from the values of the variable "ingredients_from_palm_oil_n".</strong>
</div>

In [311]:
# Data preparation
cols_to_impute = ["ingredients_from_palm_oil_n", "saturated-fat_100g"]
data_to_impute = data_bb[cols_to_impute]

In [312]:
# Replace Nan values by 0 in ingredients_from_palm_oil_n
data_bb["ingredients_from_palm_oil_n"].fillna(0.0, inplace=True)

In [313]:
data_bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85398 entries, 0 to 320770
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name                 85398 non-null  object 
 1   main_category_fr             85398 non-null  object 
 2   brands                       85398 non-null  object 
 3   nutrition_grade_fr           85398 non-null  object 
 4   fiber_100g                   85398 non-null  float64
 5   proteins_100g                85398 non-null  float64
 6   fat_100g                     85398 non-null  float64
 7   energy_100g                  85398 non-null  float64
 8   alcohol_100g                 85398 non-null  float64
 9   carbohydrates_100g           85398 non-null  float64
 10  polyols_100g                 85398 non-null  float64
 11  saturated-fat_100g           61802 non-null  float64
 12  ingredients_from_palm_oil_n  85398 non-null  float64
 13  additives_n    

In [314]:
# Create an IterativeImputer object with the LinearRegression estimator
imputer = IterativeImputer(estimator=LinearRegression())

In [315]:
# Use fit_transform method to replace missing values
data_to_impute_imputed = imputer.fit_transform(data_to_impute)

In [316]:
# Replace original column with imputed column
data_bb["saturated-fat_100g"] = data_to_impute_imputed

<ipython-input-316-4843f2213c88>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bb["saturated-fat_100g"] = data_to_impute_imputed


In [319]:
# Random value check
print("Random saturated fat :", data_bb["saturated-fat_100g"].sample(10), "\n")
print("Random ingredients from palm oil :", data_bb["ingredients_from_palm_oil_n"].sample(10), "\n")

Random saturated fat : 212679    0.070672
187569    0.000000
206479    0.000000
214854    0.000000
198490    0.000000
214937    0.000000
254697    0.000000
192499    0.000000
247064    0.070672
254084    0.044244
Name: saturated-fat_100g, dtype: float64 

Random ingredients from palm oil : 221645    0.0
224500    0.0
197925    0.0
215043    1.0
200030    0.0
67896     0.0
196339    0.0
200224    0.0
212712    0.0
213052    0.0
Name: ingredients_from_palm_oil_n, dtype: float64 



<a id='another_cell_g'></a>

#### c. <i>Buy better</i> dataframe : quantative data augmentation using fillna

In [320]:
data_bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85398 entries, 0 to 320770
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name                 85398 non-null  object 
 1   main_category_fr             85398 non-null  object 
 2   brands                       85398 non-null  object 
 3   nutrition_grade_fr           85398 non-null  object 
 4   fiber_100g                   85398 non-null  float64
 5   proteins_100g                85398 non-null  float64
 6   fat_100g                     85398 non-null  float64
 7   energy_100g                  85398 non-null  float64
 8   alcohol_100g                 85398 non-null  float64
 9   carbohydrates_100g           85398 non-null  float64
 10  polyols_100g                 85398 non-null  float64
 11  saturated-fat_100g           85398 non-null  float64
 12  ingredients_from_palm_oil_n  85398 non-null  float64
 13  additives_n    

<div class="alert-success">
☛ We can replace NaN values on our quantitative variable by 0
</div>

In [321]:
# Replace Nan values by 0 for other quantitative variables
columns_to_fill = ['additives_n','vitamin-d_100g','vitamin-c_100g']

for i in columns_to_fill:
    data_bb[i].fillna(0.0, inplace=True)

C:\Users\hp\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [322]:
data_bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85398 entries, 0 to 320770
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name                 85398 non-null  object 
 1   main_category_fr             85398 non-null  object 
 2   brands                       85398 non-null  object 
 3   nutrition_grade_fr           85398 non-null  object 
 4   fiber_100g                   85398 non-null  float64
 5   proteins_100g                85398 non-null  float64
 6   fat_100g                     85398 non-null  float64
 7   energy_100g                  85398 non-null  float64
 8   alcohol_100g                 85398 non-null  float64
 9   carbohydrates_100g           85398 non-null  float64
 10  polyols_100g                 85398 non-null  float64
 11  saturated-fat_100g           85398 non-null  float64
 12  ingredients_from_palm_oil_n  85398 non-null  float64
 13  additives_n    

<a id='another_cell_h'></a>

### D. <i>Buy better</i> : anomaly detection and treatment

- <strong>Detection of outliers using the interquartile method</strong>

In [323]:
# Interquartile range calculation
outliers = count_outliers(data_bb)
print("Number of outliers per numeric variables :", outliers)

Number of outliers per numeric variables : {'fiber_100g': 9529, 'proteins_100g': 5390, 'fat_100g': 11618, 'energy_100g': 729, 'alcohol_100g': 1755, 'carbohydrates_100g': 14618, 'polyols_100g': 293, 'saturated-fat_100g': 4464, 'ingredients_from_palm_oil_n': 3674, 'additives_n': 14877, 'vitamin-d_100g': 585, 'vitamin-c_100g': 1150}


<div class="alert-warning">
There are many outliers on our numerical variables.<br>
    ☛ <strong>We will attempt to detect and replace false values on this data.</strong>
</div>

- <strong>Detection and treatment of outliers on variables at 100g</strong>

In [324]:
# Define quantitative_var_100g
quantitative_var_100g = ['fiber_100g','proteins_100g','fat_100g','energy_100g','alcohol_100g',
                                 'carbohydrates_100g','polyols_100g','saturated-fat_100g','vitamin-d_100g','vitamin-c_100g']

In [325]:
#List of variables which contains values greater than 100 and drop
for columns in quantitative_var_100g:
    mask = data_bb[columns] > 100
    if mask.any():
        print(f"The variable {columns} contains values greater than 100")
        data_bb.drop(data_bb.loc[mask].index, inplace=True)

The variable fiber_100g contains values greater than 100
The variable fat_100g contains values greater than 100
The variable energy_100g contains values greater than 100


C:\Users\hp\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\hp\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\hp\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


<div class="alert-danger">
☛ We droped these values because it is too difficult to find a good way to replace each of them.
</div>

- <strong>Detection and treatment of negative values</strong>

In [326]:
# Define numeric_cols
numeric_cols = data_bb[['fiber_100g','proteins_100g','fat_100g','energy_100g','alcohol_100g',
                        'carbohydrates_100g','polyols_100g','saturated-fat_100g','ingredients_from_palm_oil_n',
                        'additives_n']]

for column in numeric_cols:
    if len(data_bb[column][data_bb[column] < 0]) > 0:
        print("The variable" ,column, "contains negative values.")

<div class="alert-success">
☛ We don't have any negative values in our dataframe
</div>

In [327]:
data_bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2466 entries, 774 to 320763
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name                 2466 non-null   object 
 1   main_category_fr             2466 non-null   object 
 2   brands                       2466 non-null   object 
 3   nutrition_grade_fr           2466 non-null   object 
 4   fiber_100g                   2466 non-null   float64
 5   proteins_100g                2466 non-null   float64
 6   fat_100g                     2466 non-null   float64
 7   energy_100g                  2466 non-null   float64
 8   alcohol_100g                 2466 non-null   float64
 9   carbohydrates_100g           2466 non-null   float64
 10  polyols_100g                 2466 non-null   float64
 11  saturated-fat_100g           2466 non-null   float64
 12  ingredients_from_palm_oil_n  2466 non-null   float64
 13  additives_n   

<a id='another_cell_i'></a>

### Conclusion (part 1)

<div class="alert-success">
<strong>There are 2,466 products in the final dataframe</strong> :<br>
<i>→ we have selected a few variables that may be of interest to us for the project<br>
→ we have replaced the missing values of the variable "energy_100g" using a linear regression to predict them<br>
→ we estimated the missing values of the variable "saturated-fat_100g" using the iterative imputer method from the variable "ingredients_from_palm_oil_n"<br>
→ we replaced the missing values of the other quantitative variables with 0<br>
→ we detected the outliers of our quantitative variables using the interquartile method<br>
→ we removed outliers greater than 100 for the quantitative variables "fiber_100g", "fat_100g" and "energy_100g"<br>
→ we also checked that there is no negative value
</i></div>

In [328]:
data_bb.to_csv("data_bb_final.csv", index=False)